In [1]:
%load_ext autoreload
%autoreload 2

import json

import pandas as pd
from pydash import py_

from yiutils.project_utils import find_project_root

In [2]:
proj_root = find_project_root("justfile")
print(proj_root)
data_dir = proj_root / "data"
assert data_dir.exists(), f"Data directory {data_dir} does not exist."

output_dir = proj_root / "output"
assert output_dir.exists(), f"Output directory {output_dir} does not exist."

/lus/lfs1aip1/home/b5a/ik18445.b5a/projects/llm-data-extraction


In [3]:
path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists(), (
    f"Data file {path_to_mr_pubmed_data} does not exist."
)

with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)
    mr_pubmed_df = pd.DataFrame(mr_pubmed_data)
    mr_pubmed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15860 entries, 0 to 15859
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pmid          15860 non-null  object
 1   ab            15860 non-null  object
 2   pub_date      15860 non-null  object
 3   title         15860 non-null  object
 4   journal_issn  15652 non-null  object
 5   journal       15860 non-null  object
 6   author_affil  15690 non-null  object
dtypes: object(7)
memory usage: 867.5+ KB


In [4]:
path_to_results_dir = (
    data_dir / "intermediate" / "llm-results" / "isb-ai-111997" / "results"
)
assert path_to_results_dir.exists()
json_files = list(path_to_results_dir.glob("*.json"))
print(len(json_files))

json_data = []
for json_file in json_files:
    with open(json_file, "r") as f:
        data = {"data": json.load(f), "filename": str(json_file.name)}
        json_data.append(data)

results_df = pd.concat(
    [
        pd.DataFrame(data["data"]).assign(filename=data["filename"])
        for data in json_data
    ],
).reset_index(drop=True)
tmp_output_file = output_dir / "tmp_results.csv"
results_df.to_csv(tmp_output_file, index=False)
results_df.info()
results_df

70
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7070 entries, 0 to 7069
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                7070 non-null   object
 1   ab                  7070 non-null   object
 2   pub_date            7070 non-null   object
 3   title               7070 non-null   object
 4   journal_issn        7012 non-null   object
 5   journal             7070 non-null   object
 6   author_affil        7002 non-null   object
 7   metadata            6980 non-null   object
 8   metainformation     741 non-null    object
 9   results             7070 non-null   object
 10  resultsinformation  7069 non-null   object
 11  exposures           90 non-null     object
 12  outcomes            90 non-null     object
 13  methods             92 non-null     object
 14  population          71 non-null     object
 15  filename            7070 non-null   object
dtypes: object(16)
memory 

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,metainformation,results,resultsinformation,exposures,outcomes,methods,population,filename
0,38025973,Numerous investigations have revealed the inte...,2023-06-30,Investigating the gut microbiota's influence o...,2516-1571,Precision clinical medicine,"Department of Dermatology, Hunan Engineering R...",{},"{'error': 'Failed Expecting ',' delimiter: lin...",{},"{'error': 'Failed Expecting ',' delimiter: lin...",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json
1,38025156,To investigate the causal association between ...,2023-07-10,Causal association between subtypes of osteoar...,2665-9131,Osteoarthritis and cartilage open,"Academic Rheumatology, Clinical Sciences Build...","{'exposures': [{'id': '1', 'trait': 'Osteoarth...",NaN,"[{'exposure': 'Hip OA', 'outcome': 'Multisite ...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json
2,38025107,The interactions between fasting insulin level...,2023-11-08,High blood pressure mediated the effect of fas...,2055-2076,Digital health,"Department of Gastroenterology, Liyuan Hospita...","{'exposures': [{'id': '1', 'trait': 'Fasting i...",NaN,"[{'exposure': 'Fasting insulin levels', 'outco...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json
3,38023262,The high incidence of breast cancer (BC) promp...,2023-06-24,Prognostic value analysis of cholesterol and c...,2234-943X,Frontiers in oncology,"Department of Breast Surgery, The First Affili...",{},{'error': 'Failed Expecting value: line 12 col...,{},{'error': 'Failed Expecting value: line 12 col...,NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json
4,38022630,Although numerous observational studies have i...,2023-07-30,Association between genetically predicted rheu...,1664-3224,Frontiers in immunology,"Department of Plastic Surgery, Longyan First A...",NaN,{'error': 'Some methods do not match those lis...,"[{'exposure': 'Rheumatoid arthritis (RA)', 'ou...","{'error': None, 'explanation': None}","[{'id': '1', 'trait': 'Rheumatoid arthritis (R...","[{'id': '1', 'trait': 'Alopecia areata (AA)', ...","[inverse variance-weighted (IVW) method, weigh...",[large-scale gene aggregation data from genome...,mr_extract_llama3_sample_array_18.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7065,32628676,To efficiently transform genetic associations ...,2019-06-24,Linking protein to phenotype with Mendelian Ra...,1553-7404,PLoS genetics,"MRC Human Genetics Unit, Institute of Genetics...","{'exposures': [{'id': '1', 'trait': 'SHPS1 (SI...",NaN,"[{'exposure': 'SHPS1 (SIRPA)', 'outcome': 'sch...",{'error': 'No numeric values provided for beta...,NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json
7066,32627641,Background Elevated blood pressure is a major ...,2020-07-07,Genetically Predicted Midlife Blood Pressure a...,2047-9980,Journal of the American Heart Association,Department of Epidemiology and Biostatistics S...,"{'exposures': [{'id': '1', 'trait': 'Midlife b...",NaN,[{'exposure': 'Genetically predicted mean arte...,"{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json
7067,32610558,Testosterone has historically been linked to s...,2020-06-08,The Potential Effect of Aberrant Testosterone ...,2073-4425,Genes,"Bio-X Institutes, Key Laboratory for the Genet...",{},{'error': 'Failed Expecting value: line 48 col...,{},{'error': 'Failed Expecting value: line 48 col...,NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json
7068,32592749,The effect of body mass index (BMI) on the ris...,2020-05-06,Elevated body mass index is associated with an...,1469-0691,Clinical microbiology and infection : the offi...,"Lady Davis Institute, Jewish General Hospital,...","{'exposures': [{'id': '1', 'trait': 'Body mass...",NaN,"[{'exposure': 'Body mass index (BMI)', 'outcom...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3

# basic diagnostics

In [23]:
# num missing
print(results_df["pmid"].isna().sum(), results_df.shape[0])
# num duplicates
print(results_df["pmid"].drop_duplicates().shape[0], results_df["pmid"].shape[0])


0 7070
7001 7070


# results summary

# weird examples

In [24]:
weird_df = results_df[~results_df["exposures"].isna()]
weird_df.info()
weird_df

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 4 to 7012
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                90 non-null     object
 1   ab                  90 non-null     object
 2   pub_date            90 non-null     object
 3   title               90 non-null     object
 4   journal_issn        90 non-null     object
 5   journal             90 non-null     object
 6   author_affil        85 non-null     object
 7   metadata            0 non-null      object
 8   metainformation     90 non-null     object
 9   results             90 non-null     object
 10  resultsinformation  90 non-null     object
 11  exposures           90 non-null     object
 12  outcomes            90 non-null     object
 13  methods             90 non-null     object
 14  population          67 non-null     object
 15  filename            90 non-null     object
dtypes: object(16)
memory usage: 12.

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,metainformation,results,resultsinformation,exposures,outcomes,methods,population,filename
4,38022630,Although numerous observational studies have i...,2023-07-30,Association between genetically predicted rheu...,1664-3224,Frontiers in immunology,"Department of Plastic Surgery, Longyan First A...",NaN,{'error': 'Some methods do not match those lis...,"[{'exposure': 'Rheumatoid arthritis (RA)', 'ou...","{'error': None, 'explanation': None}","[{'id': '1', 'trait': 'Rheumatoid arthritis (R...","[{'id': '1', 'trait': 'Alopecia areata (AA)', ...","[inverse variance-weighted (IVW) method, weigh...",[large-scale gene aggregation data from genome...,mr_extract_llama3_sample_array_18.json
88,37975615,Although a large number of existing studies ha...,2023-10-10,Exploring genetic associations between vitilig...,1600-0625,Experimental dermatology,"Department of Dermatology, The Second Affiliat...",NaN,{'error': 'Some methods do not match those lis...,[],{'error': 'No significant causal associations ...,"[{'id': '1', 'trait': 'Vitiligo', 'category': ...","[{'id': '1', 'trait': 'Depression', 'category'...","[{'id': '1', 'method': 'Inverse-variance weigh...",NaN,mr_extract_llama3_sample_array_18.json
187,37651993,Non-linear Mendelian randomization is an exten...,2022-12-12,Violation of the Constant Genetic Effect Assum...,1423-0062,Human heredity,"MRC Biostatistics Unit, University of Cambridg...",NaN,{'error': 'Some methods do not match those lis...,"[{'exposure': '25-hydroxyvitamin D (25(OH)D)',...","{'error': 'No results provided in abstract', '...","[{'id': '1', 'trait': '25-hydroxyvitamin D (25...","[{'id': '1', 'trait': 'All-cause mortality', '...","[non-linear Mendelian randomization, residual ...",[low vitamin D individuals],mr_extract_llama3_sample_array_23.json
205,37390109,Non-linear Mendelian randomization is an exten...,2022-12-26,Relaxing parametric assumptions for non-linear...,1553-7404,PLoS genetics,"MRC Biostatistics Unit, School of Clinical Med...",NaN,{'error': 'No information on population is pro...,"[{'exposure': 'alcohol intake', 'outcome': 'sy...","{'error': 'Some fields are missing', 'explanat...","[{'id': '1', 'trait': 'Alcohol intake', 'categ...","[{'id': '1', 'trait': 'Systolic blood pressure...",[non-linear Mendelian randomization],NaN,mr_extract_llama3_sample_array_27.json
441,38502664,Multiple Sclerosis (MS) is an immune-related d...,2023-07-10,Association between multiple sclerosis and can...,1932-6203,PloS one,"VIP Inpatient Department, State Key Laboratory...",NaN,{'error': 'Some methods do not match those lis...,"[{'exposure': 'Multiple Sclerosis (MS)', 'outc...","{'error': None, 'explanation': None}","[{'id': '1', 'trait': 'Multiple Sclerosis (MS)...","[{'id': '1', 'trait': 'cancer', 'category': 'n...","[two-sample MR method, Inverse variance weight...",[International Multiple Sclerosis Genetics Con...,mr_extract_llama3_sample_array_7.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,38335403,We conducted a study to evaluate the impact of...,2024-02-10,Mendelian randomization to evaluate the effect...,1536-5964,Medicine,"Department of Neurology, General Hospital of N...",NaN,{'error': 'Some methods do not match those lis...,"[{'exposure': 'folic acid supplementation', 'o...","{'error': None, 'explanation': None}","[{'id': '1', 'trait': 'Folic acid supplementat...","[{'id': '1', 'trait': 'Alzheimer disease (AD)'...","[Mendelian randomization (MR) analysis, Invers...",[No specific population mentioned in the abstr...,mr_extract_llama3_sample_array_11.json
6741,37576893,[This corrects the article DOI: 10.3389/fonc.2...,2023-07-06,Corrigendum: Diseases of the musculoskeletal s...,2234-943X,Frontiers in oncology,"Department of Radiotherapy, The First Affiliat...",NaN,"{'error': 'No exposures, outcomes, or methods ...",[],"{'error': 'No results provided in abstract', '...",[],[],[],[No population information provided in